In [ ]:
!pip install git+https://github.com/edtechre/pybroker.git

In [ ]:
!pip install akshare numpy pandas

In [4]:
import pybroker
print(pybroker.__version__)  # 若无版本号，说明安装可能有问题
!python --version

1.2.10
Python 3.11.12


In [6]:
from pybroker import Strategy, StrategyConfig
from pybroker.ext.data import AKShare
import pandas as pd

# 策略逻辑：收盘价上穿/下穿5日均线
def handle_data(ctx):
    symbol = ctx.symbol
    close = ctx.close

    if len(close) < 6:
        return

    # 使用 Pandas 计算 5 日均线
    ma5_series = pd.Series(close).rolling(window=5).mean()
    ma5 = ma5_series.values

    # 获取持仓情况（注意 pos_type 应为 "long"）
    position = ctx.pos(symbol, "long")
    has_position = position and position.shares > 0

    # 上穿5日均线，买入
    if close[-1] > ma5[-1] and close[-2] <= ma5[-2]:
        if not has_position:
            ctx.buy_shares = 100

    # 下穿5日均线，卖出
    elif close[-1] < ma5[-1] and close[-2] >= ma5[-2]:
        if has_position:
            ctx.sell_all_shares()

# 初始化数据源
data_source = AKShare()

# 创建策略对象
strategy = Strategy(
    data_source,
    start_date="2023-01-01",
    end_date="2023-12-31",
    config=StrategyConfig(initial_cash=20000)
)

# 添加策略逻辑
strategy.add_execution(handle_data, symbols=["601318.SH"])  # 中国平安（上交所代码）

# 执行回测
result = strategy.backtest()


Backtesting: 2023-01-01 00:00:00 to 2023-12-31 00:00:00

Loading bar data...
Loaded bar data: 0:00:01 

Test split: 2023-01-03 00:00:00 to 2023-12-29 00:00:00


100% (242 of 242) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:01
总收益:                       name                value
0              trade_count                   28
1     initial_market_value              20000.0
2         end_market_value              19466.0
3                total_pnl               -540.0
4           unrealized_pnl                  6.0
5         total_return_pct                 -2.7
6             total_profit                674.0
7               total_loss              -1214.0
8               total_fees                  0.0
9             max_drawdown               -774.0
10        max_drawdown_pct            -3.839667
11       max_drawdown_date  2023-12-13 00:00:00
12                win_rate            32.142857
13               loss_rate            67.857143
14          winning_trades                    9
15           losing_trades                   19
16                 avg_pnl           -19.285714
17          avg_return_pct            -0.405357
18          avg_trade_bars             3.035714
19     

In [10]:
# 输出结果
print("总收益:", result.metrics_df[result.metrics_df['name'] == 'total_pnl'])
print("交易记录:\n", result.trades)

总收益:         name  value
3  total_pnl -540.0
交易记录:
     type     symbol entry_date  exit_date  entry   exit  shares    pnl  \
id                                                                       
1   long  601318.SH 2023-01-12 2023-01-18  50.77  51.52     100   75.0   
2   long  601318.SH 2023-01-30 2023-01-31  52.54  51.56     100  -98.0   
3   long  601318.SH 2023-02-10 2023-02-16  49.28  49.52     100   24.0   
4   long  601318.SH 2023-02-21 2023-02-23  49.78  49.26     100  -52.0   
5   long  601318.SH 2023-03-02 2023-03-07  50.09  48.48     100 -161.0   
6   long  601318.SH 2023-03-20 2023-03-21  46.07  46.09     100    2.0   
7   long  601318.SH 2023-03-22 2023-03-27  46.50  45.92     100  -58.0   
8   long  601318.SH 2023-04-04 2023-04-07  45.54  45.46     100   -8.0   
9   long  601318.SH 2023-04-10 2023-04-13  45.89  45.09     100  -80.0   
10  long  601318.SH 2023-04-18 2023-04-24  48.10  46.98     100 -112.0   
11  long  601318.SH 2023-04-28 2023-05-10  51.46  52.98     